In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend

machine = "H1-1SC"
backend = QuantinuumBackend(device_name=machine)
backend.login()

In [ ]:
# #### Set up Circuit
#
# Set up the RUS circuit. This involves 2 steps:
# 1. Set up Wasm File handler. This checks the Wasm file and can list the available functions within the file.
# 2. Set up the pytket circuit, which calls the Wasm file handler.


import pathlib
from pytket import wasm

rus_dir = pathlib.Path().cwd().joinpath("RUS_WASM")
wasm_file = rus_dir.joinpath("rus_wasm.wasm")
wfh = wasm.WasmFileHandler(wasm_file)

print(repr(wfh))

In [ ]:
from pytket.circuit import Circuit, Qubit
from pytket.circuit.logic_exp import reg_lt


def build_rus_circuit(n_repetitions: int, cond_execute: int) -> Circuit:
    """
    n_repetitions (int): number of attempts
    cond_execute (int): condition to execute, i.e. run this until a total of cond_execute |1> measurements is achieved.

    """
    # Create circuit with two qubits
    circuit = Circuit(2, name=f"RUS_limit={n_repetitions}_cond={cond_execute}")

    # Add classical registers
    creg0 = circuit.add_c_register("creg0", 1)
    creg1 = circuit.add_c_register("creg1", 1)
    cond = circuit.add_c_register("cond", 32)
    count = circuit.add_c_register("count", 32)

    # Set cond to 0
    circuit.add_c_setreg(0, cond)

    # Loops
    for loop_iter in range(1, n_repetitions + 1):

        circuit.H(0, condition=reg_lt(cond, cond_execute))
        circuit.CX(0, 1, condition=reg_lt(cond, cond_execute))
        circuit.Measure(Qubit(1), creg1[0], condition=reg_lt(cond, cond_execute))

        # Add wasm call with the parameters creg1 and count, writing the result to cond
        # The function "add_count" is used from the Wasm file
        circuit.add_wasm_to_reg(
            "add_count",
            wfh,
            [creg1, count],
            [cond],
            condition=reg_lt(cond, cond_execute),
        )

        circuit.add_c_setreg(loop_iter, count, condition=reg_lt(cond, cond_execute))
        circuit.Reset(0, condition=reg_lt(cond, cond_execute))

    circuit.Measure(Qubit(0), creg0[0])
    circuit.Measure(Qubit(1), creg1[0])

    return circuit

In [ ]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=0)

n_shots = 1000
backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker="H1-1SC")
handle = backend.process_circuit(
    compiled_circuit, n_shots=n_shots, wasm_file_handler=wfh
)
result = backend.get_result(handle)
print(result)

import json

with open(compiled_circuit.name + ".json", "w", encoding="utf-8") as file:
    json.dump(result.to_dict(), file)


from pytket.unit_id import Bit
from pytket.backends.backendresult import BackendResult


def get_creg1_counts_from_rus_result(
    result_dict: dict,
) -> tuple[dict[str, BackendResult]]:
    """Analyze results of RUS experiment."""
    creg1_0_cnts = {}
    creg1_1_cnts = {}

    for key, res in result_dict.items():
        creg1_counts = res.get_counts(cbits=[Bit("creg1", 0)])
        creg1_0_cnts[key] = creg1_counts[(0,)]
        creg1_1_cnts[key] = creg1_counts[(1,)]

    return creg1_0_cnts, creg1_1_cnts


result1 = {}
result1[compiled_circuit.name] = result
creg1_0_cnts, creg1_1_cnts = get_creg1_counts_from_rus_result(result1)
print(creg1_0_cnts, creg1_1_cnts)


In [ ]:
def get_rus_circuit_batch(
    machine_name: str, limit_start: int, limit_end: int, cond_execute: int
) -> list[Circuit]:
    backend = QuantinuumBackend(device_name=machine_name)

    circuit_list = []

    for limit in range(limit_start, limit_end + 1):
        rus_circuit = build_rus_circuit(limit, cond_execute)
        compiled_rus_circuit = backend.get_compiled_circuit(rus_circuit)
        circuit_list.append(compiled_rus_circuit)

    return circuit_list


def get_batch_cost(
    compiled_circuit_list: list[Circuit], machine: str, n_shots: int
) -> float:

    syntax_checker = machine.replace("E", "SC")

    total_cost = 0
    for compiled_circuit in compiled_circuit_list:
        circuit_cost = backend.cost(
            circuit=compiled_circuit, syntax_checker=syntax_checker, n_shots=n_shots
        )
        if circuit_cost is not None:
            total_cost += circuit_cost

    return total_cost


def run_rus_experiment(
    machine: str,
    n_shots: int,
    max_batch_cost: int,
    cond_execute: int,
    limit_start: int,
    limit_end: int,
) -> tuple[dict, float]:

    backend = QuantinuumBackend(device_name=machine)
    rus_circuit_list = get_rus_circuit_batch(
        machine, limit_start, limit_end, cond_execute
    )

    experiment_cost: float = get_batch_cost(rus_circuit_list, machine, n_shots)

    batch_handles = {}

    batch_start = backend.start_batch(
        max_batch_cost,
        rus_circuit_list[0],
        n_shots,
        wasm_file_handler=wfh,
    )
    batch_handles[rus_circuit_list[0].name] = batch_start

    for compiled_circuit in rus_circuit_list[1:-2]:
        batch_mid = backend.add_to_batch(
            batch_start,
            compiled_circuit,
            n_shots=n_shots,
            wasm_file_handler=wfh,
        )
        batch_handles[compiled_circuit.name] = batch_mid

    batch_end = backend.add_to_batch(
        batch_start,
        rus_circuit_list[-1],
        n_shots=n_shots,
        wasm_file_handler=wfh,
        batch_end=True,
    )
    batch_handles[rus_circuit_list[-1].name] = batch_end

    return batch_handles, experiment_cost